<a href="https://colab.research.google.com/github/rfclara/fa_xhosa/blob/main/evaluation_asr_xhosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Are evangelist speech models able to understand natural language ?

# Introduction

In this notebook, we explore the performance of an existing Automatic Speech Recognition (ASR) model for Xhosa, developed under the [Massively Multilingual Speech (MMS)](https://huggingface.co/docs/transformers/model_doc/mms) project by Facebook AI Research. The model's evaluation is conducted using a natural data corpus provided by the University of Gothenburg's Spraakbanken (Language Bank), accessible at [Spraakbanken's Xhosa Corpus](https://spraakbanken.gu.se/korp/?mode=xhosa#?lang=eng&cqp=%5B%5D&corpus=xhosa).

## Objectives

1. **Align and preprocess the data:**
   - The original Spraakbanken's Xhosa Corpus is fully manually transcribed by students in South Africa who have undergone specific training. Each sentence has been aligned to the corresponding audio using the MMS [forced alignment tool](https://github.com/facebookresearch/fairseq/tree/main/examples/mms/data_prep), as detailed in this [Notebook](https://github.com/rfclara/fa_xhosa/blob/main/xhosa_forced_alignement.ipynb). The preprocessing involved removing punctuation and comments such as \<laugh> or \<code-switching> to ensure the corpus is in a consistent format suitable for evaluation.

2. **Evaluate Existing MMS ASR Model:**
  - Assess the robustness and accuracy of the MMS ASR model for Xhosa using a natural language dataset from Spraakbanken. Thanks to this natural data, we will have the opportinity to verify Meta's claim: "while the content of the audio recordings is religious, our analysis shows that this does not overly bias the model to produce more religious language”.


3. **Fine-Tuning the ASR Model:**
   - Investigate whether fine-tuning the MMS ASR model on the natural data corpus improves its performance, given that the original model appears to be primarily trained on biblical texts.

4. **Compare the performances:**
Compare WER and CER metrix for the MMS ASR model before and after fine-tunning.

## Structure of the Notebook

1. **Data Preparation:**
   - Steps to download and preprocess the dataset.
   
2. **Model Evaluation:**
   - Application of the MMS ASR model on the prepared dataset.
   - Performance metrics and error analysis.
   
3. **Fine-Tuning:**
   - Process of fine-tuning the MMS ASR model using the natural data corpus.
   - Comparison of performance metrics before and after fine-tuning.
   
4. **Results and Discussion:**
   - Insights gained from the evaluation.
   - Discussion on the model's strengths and areas for improvement.
   
5. **Conclusion:**
   - Summary of findings.
   - Future work and potential improvements.

By following this structured approach, we aim to provide a comprehensive assessment of the MMS ASR model's capabilities in handling real-world Xhosa speech data, contributing valuable insights to the ongoing development and refinement of multilingual ASR technologies.


# Step 1: Set Up the Environment
Install required libraries and import the requiered packages

In [ ]:
!pip install torchaudio transformers jiwer
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import os
import json

In [2]:
!git clone https://github.com/pytorch/fairseq
!pwd
%cd "/content/fairseq"
!pip install --editable ./
!pip install tensorboardX

  Building editable for fairseq (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-0.editable-cp310-cp310-linux_x86_64.whl size=9543 sha256=bc3c18189d35233dbffe987ce0a19dc9891378cbddb421f92c6be6290c3f4dda
  Stored in directory: /tmp/pip-ephem-wheel-cache-td0v9qz0/wheels/c6/d7/db/bc419b1daa8266aa8de2a7c4d29f62dbfa814e8701fe4695a2
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141211 sha256=1812311aa1ef8a2a749d9580cc1fb569f2daf64c6f9ce8c1e04b368a153d2342
  Stored in directory: /root/.cache/pip/wheels/a7/20/bd/e1477d664f22d99989fd28ee1a43d6633dddb5cb9e801350d5
Successfully built fairseq antlr4-python3-runtime
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.5 MB/s eta 0:00:00


## Step 2: Load the Model
Load the processor and model for Xhosa from Facebook's MMS

In [ ]:
from transformers import Wav2Vec2ForCTC, AutoProcessor

model_id = "facebook/mms-1b-all" # mms-1b-fl102, mms-1b-l1107, mms-1b-all
target_lang = "xho"  # Xhosa language code

processor = AutoProcessor.from_pretrained(model_id, target_lang=target_lang)
model = Wav2Vec2ForCTC.from_pretrained(model_id, target_lang=target_lang, ignore_mismatched_sizes=True)

# Step 3: Download the clean and aligned audio files and gold transcriptions

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/aligned.zip /content
!unzip /content/aligned.zip -d /

Mounted at /content/drive
Archive:  /content/aligned.zip
   creating: /content/xhosa/aligned/
   creating: /content/xhosa/aligned/BU160331M_a/
  inflating: /content/xhosa/aligned/BU160331M_a/segment2.flac  
  inflating: /content/xhosa/aligned/BU160331M_a/segment6.flac  
  inflating: /content/xhosa/aligned/BU160331M_a/manifest.json  
  inflating: /content/xhosa/aligned/BU160331M_a/segment1.flac  
  inflating: /content/xhosa/aligned/BU160331M_a/segment4.flac  
  inflating: /content/xhosa/aligned/BU160331M_a/segment3.flac  
  inflating: /content/xhosa/aligned/BU160331M_a/segment5.flac  
  inflating: /content/xhosa/aligned/BU160331M_a/segment0.flac  
   creating: /content/xhosa/aligned/BLN150925D_b/
 extracting: /content/xhosa/aligned/BLN150925D_b/segment8.flac  
 extracting: /content/xhosa/aligned/BLN150925D_b/segment2.flac  
 extracting: /content/xhosa/aligned/BLN150925D_b/segment6.flac  
 extracting: /content/xhosa/aligned/BLN150925D_b/segment18.flac  
 extracting: /content/xhosa/aligne

# Step 4: Prediction


In [5]:
def load_and_process_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    waveform = waveform.squeeze()  # Remove channel dimension if it's mono
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
    return waveform

# Step 4: Iterate through directories and process each manifest file
base_path = "/content/xhosa/aligned"
all_transcriptions = []
all_references = []

for root, dirs, files in os.walk(base_path):
    for file in files:
        if file == "manifest.json":
            manifest_path = os.path.join(root, file)
            with open(manifest_path, 'r') as f:
                for line in f:
                    entry = json.loads(line.strip())
                    audio_path = entry["audio_filepath"]
                    reference_text = entry["normalized_text"]
                    if not reference_text.strip():
                      continue
                    # Load and process the audio file
                    waveform = load_and_process_audio(audio_path)

                    # Process the audio input
                    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)

                    # Ensure input is 2D [batch_size, sequence_length]
                    if inputs.input_values.dim() == 3 and inputs.input_values.size(1) == 2:
                        inputs.input_values = inputs.input_values.mean(dim=1)  # Convert stereo to mono

                    # Perform inference
                    with torch.no_grad():
                        logits = model(inputs.input_values).logits

                    # Decode the predicted IDs to text
                    predicted_ids = torch.argmax(logits, dim=-1)
                    transcription = processor.batch_decode(predicted_ids)[0]

                    # Collect transcriptions and references
                    all_transcriptions.append(transcription)
                    all_references.append(reference_text)

                    # Print the result for each segment
                    print(f"Reference: {reference_text}")
                    print(f"Prediction: {transcription}")

                    print()

Transcription: dingunosabelo futhu
Reference: ndingunosabelo futu

Transcription: umnyeni wam ngubhakhoni futhu
Reference: umyeni wam ngubhakhoni futu

Transcription: uphangela erustenburg e-min yase impala e-14cf
Reference: uphangela erustenburg emayini yaseimpala e14 charles

Transcription: kukude kakhulu ukusuka apha exhorha
Reference: kukude kakhulu ukusuka apha exhorha

Transcription: ubuya kathathu ngonyaka
Reference: ubuya kathathu ngonyaka

Transcription: ubuya ngepasika abuye ngojuni aphinde abuye ngodisemba qha
Reference: ubuya ngepasika abuye ngojuni aphinde abuye ngodisemba qha

Transcription: kunqabile ukudibana naye
Reference: kunqabile ukudibana naye

Transcription: abeenaboti utii
Reference: yhena bhuti uthini

Transcription: dkukandona akakadolunza
Reference: uthi kuqhubeka ntoni na kwadalumzi

Transcription: d
Reference: uthini

Transcription: alokudalonzi nanononyana ukuqale emfaza nkwala
Reference: kaloku udalumzi unonyana unonyana woqala emfazini wakhe

Transcripti

# Step 5: Evaluation

*   WER
*   CER



In [6]:
!pip install jiwer

In [23]:
import jiwer
from jiwer import wer, cer
wer = wer(all_references, all_transcriptions)
cer = cer(all_references, all_transcriptions)
print("Word Error Rate (WER):", wer)
print(f"Character Error Rate (CER): {cer}")

Word Error Rate (WER): 0.8152631578947368
Character Error Rate (CER): 0.3098368087035358


In [21]:
import difflib
from IPython.core.display import display, HTML

def highlight_differences(correct: str, user: str) -> str:
    differ = difflib.Differ()
    diff = list(differ.compare(correct, user))

    result = []
    for part in diff:
        if part.startswith('  '):
            result.append(part[2:])  # No difference
        elif part.startswith('- '):
            result.append(f'<span style="color:red;">{part[2:]}</span>')  # Incorrect character
        elif part.startswith('+ '):
            result.append(f'<span style="color:green;">{part[2:]}</span>')  # Additional character
        elif part.startswith('? '):
            pass  # Ignore these lines

    return ''.join(result)

for i in range(len(all_references)):
  correct_answer = all_references[i]
  user_answer = all_transcriptions[i]
  highlighted = highlight_differences(correct_answer, user_answer)
  print("model:")
  print(all_transcriptions[i])
  display(HTML(f"<div style='font-family:monospace;'>{highlighted}</div>"))
  print("human:")
  print(all_references[i])
  print()


model:
dingunosabelo futhu


human:
ndingunosabelo futu

model:
umnyeni wam ngubhakhoni futhu


human:
umyeni wam ngubhakhoni futu

model:
uphangela erustenburg e-min yase impala e-14cf


human:
uphangela erustenburg emayini yaseimpala e14 charles

model:
kukude kakhulu ukusuka apha exhorha


human:
kukude kakhulu ukusuka apha exhorha

model:
ubuya kathathu ngonyaka


...

human:
iqale ngo2013 izophela ngooctober kulo nyaka

model:
onyakuzayo plana a ukuye sikoloa


human:
uhm kulo nyaka uzayo ndiplana ukuy a esikolweni

model:
isazojonga amayunivesithi awwenze azohambelana lendifuna e-agriculture


human:
ndisazojonga amayunivesithi awenza azohambelana nale ndiyifunayo iagriculture

model:
bendifuna kukakho iamakholethi nacmiyunivesithi


human:
bendifuna ukuqala i amacollege namayunivesithi eeh

model:



human:
ohh oh

